# Extension Project 
## Part 1: Extending the Analysis of AGN to include the X-ray band.
This script is intended to provide an extension of the work done in the first script. This extension in it's current form as no real aim and is intended to be exploratory. 


Notes for Project Extension 
* Finish off by fixing the Donley Selection
    * If time permits add the Stern Selection
* Maybe crossmatch with X-Ray data?
    * Focus on one field, CDFS with XMM-Newton data
    * chandra space 4Ms or 7Ms survey in topcat
    * Make liberal use of the crossmatching feature in TOPCAT or similarly follow on with this project and use the crossmatching feature in python
    * Use the XMM-Newton data to find the X-ray fluxes of the AGN
    * Use the seleciton criteria in Cowley et al. 2016 to select AGN by Luminosity and Hardness Ratio.
* Xray selection
* AGN Selection using luminosity threshold (CDFS)
* Possible UVJ selection
    * Potentially contaiminated by dusty torus
    * What we could do would be to first plot the UVJ diagram and see how the distribution looks in different redshift bins
    * Then we could plot the UVJ diagram for the AGN selected by the X-ray luminosity and see how that compares to the general population
    * we could also compare this to the IR selected AGNS, maybe using a subset of these AGN that are also X-ray selected.
* Could also look at how star-formation rate is effected between xray agn or ir agn?


HR <- Crossmatch the the HR from the XRAY ZFourge catalogue.



# Cross Matching ZFOURGE with XMM-Newton Data
* Focus on one field, CDFS with XMM-Newton data
* chandra space 4Ms survey in topcat (Check Paper)
    * Use the XMM-Newton data to find the X-ray fluxes of the AGN
* Make liberal use of the crossmatching feature in TOPCAT or similarly follow on with this project and create a cross-matching function in python
* Use the seleciton criteria in Cowley et al. 2016 to select AGN by Luminosity and Hardness Ratio.

In [1]:
# Import all the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy as ap
import os

In [2]:
# Attempt to read in the fits file of the CDFS 7Ms survey


In [1]:
# # Import the data in the fields
# zfourge_fields = {
#     'CDFS': ['cdfs/cdfs.v1.6.11.cat', 'cdfs/cdfs.v1.6.9.zout', 'cdfs/cdfs.v1.6.9.sfr.v0.4.cat','cdfs/cdfs.v1.6.9.rest.v0.9.cat','cdfs/cdfs.v1.6.9.xray.v0.4.cat'],
#     'COSMOS': ['cosmos/cosmos.v1.3.8.cat', 'cosmos/cosmos.v1.3.6.zout', 'cosmos/cosmos.v1.3.6.sfr.v0.4.cat', 'cosmos/cosmos.v1.3.6.rest.v0.9.cat','cosmos/cosmos.v1.3.6.xray.v0.4.cat'],
#     'UDS': ['uds/uds.v1.5.10.cat', 'uds/uds.v1.5.8.zout', 'uds/uds.v1.5.8.sfr.v0.4.cat', 'uds/uds.v1.5.8.rest.v0.9.cat', 'uds/uds.v1.5.8.xray.v0.4.cat']
# }

# field = 'COSMOS'

# # Construct file paths using os.path.join() to make it platform-independent
# catalog_file = os.path.join(zfourge_fields[field][0])
# eazy_file = os.path.join(zfourge_fields[field][1])
# sfr_file = os.path.join(zfourge_fields[field][2])
# #flux_file = os.path.join(zfourge_fields[field][3])
# #xray_file = os.path.join(zfourge_fields[field][4])








# # Read the files into DataFrames
# df = pd.DataFrame(np.genfromtxt(catalog_file, names=True, dtype=None, encoding='ascii'))
# eazy_df = pd.DataFrame(np.genfromtxt(eazy_file, names=True, dtype=None, encoding='ascii'))
# sfr_df = pd.DataFrame(np.genfromtxt(sfr_file, names=True, dtype=None, encoding='ascii'))
# # flux_df = pd.DataFrame(np.genfromtxt(flux_file, names=True, dtype=None, encoding='ascii'))
# # xray_df = pd.DataFrame(np.genfromtxt(xray_file, names=True, dtype=None, encoding='ascii'))


NameError: name 'os' is not defined

KeyError: 0